In [85]:
# import libraries
import pandas as pd
import numpy as np
import re

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

import joblib

In [86]:
data = pd.read_csv("D:/D/Machine Learning/data/Spotify/spotify_songs.csv")
data.head(3)

,track_id,track_name,track_artist,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,playlist_genre,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,mood
0,6f807x0ima9a1j3VPbc7VN,I Don't Care (with Justin Bieber) - Loud Luxur...,Ed Sheeran,66,2oCs0DGTsRO98Gh5ZSl2Cx,I Don't Care (with Justin Bieber) [Loud Luxury...,14-06-2019,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,-2.634,1,0.0583,0.1020,0.000000,0.0653,0.518,122.036,194754,Happy
1,0r7CVbZTWZgbTCYdfa2P31,Memories - Dillon Francis Remix,Maroon 5,67,63rPSO264uRjW1X5E6cWv6,Memories (Dillon Francis Remix),13-12-2019,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,-4.969,1,0.0373,0.0724,0.004210,0.3570,0.693,99.972,162600,Happy
2,1z1Hg7Vb0AhHDiEmnDE79l,All the Time - Don Diablo Remix,Zara Larsson,70,1HoSmj2eLcsrR0vE9gThr4,All the Time (Don Diablo Remix),05-07-2019,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,-3.432,0,0.0742,0.0794,0.000023,0.1100,0.613,124.008,176616,Happy


In [87]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32833 entries, 0 to 32832
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   track_id                  32833 non-null  object 
 1   track_name                32828 non-null  object 
 2   track_artist              32828 non-null  object 
 3   track_popularity          32833 non-null  int64  
 4   track_album_id            32833 non-null  object 
 5   track_album_name          32828 non-null  object 
 6   track_album_release_date  32833 non-null  object 
 7   playlist_name             32833 non-null  object 
 8   playlist_id               32833 non-null  object 
 9   playlist_genre            32833 non-null  object 
 10  playlist_subgenre         32833 non-null  object 
 11  danceability              32833 non-null  float64
 12  energy                    32833 non-null  float64
 13  key                       32833 non-null  int64  
 14  loudne

In [88]:
# check null values inside the data
print(data.isnull().sum())
data = data.dropna()

track_id                    0
track_name                  5
track_artist                5
track_popularity            0
track_album_id              0
track_album_name            5
track_album_release_date    0
playlist_name               0
playlist_id                 0
playlist_genre              0
playlist_subgenre           0
danceability                0
energy                      0
key                         0
loudness                    0
mode                        0
speechiness                 0
acousticness                0
instrumentalness            0
liveness                    0
valence                     0
tempo                       0
duration_ms                 0
mood                        0
dtype: int64


In [89]:
data.dtypes

track_id                     object
track_name                   object
track_artist                 object
track_popularity              int64
track_album_id               object
track_album_name             object
track_album_release_date     object
playlist_name                object
playlist_id                  object
playlist_genre               object
playlist_subgenre            object
danceability                float64
energy                      float64
key                           int64
loudness                    float64
mode                          int64
speechiness                 float64
acousticness                float64
instrumentalness            float64
liveness                    float64
valence                     float64
tempo                       float64
duration_ms                   int64
mood                         object
dtype: object

In [90]:
# Categorized column based on their data type
column_object = data.select_dtypes(include=['object']).columns
column_int = data.select_dtypes(include=['int64']).columns
column_float = data.select_dtypes(include=['float64']).columns

print(column_object)
print(column_int)
print(column_float)

Index(['track_id', 'track_name', 'track_artist', 'track_album_id',
       'track_album_name', 'track_album_release_date', 'playlist_name',
       'playlist_id', 'playlist_genre', 'playlist_subgenre', 'mood'],
      dtype='object')
Index(['track_popularity', 'key', 'mode', 'duration_ms'], dtype='object')
Index(['danceability', 'energy', 'loudness', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo'],
      dtype='object')


In [91]:
# Handlke date
data['track_album_release_date'] = pd.to_datetime(data['track_album_release_date'], format='%d-%m-%Y')
data['date'] = data['track_album_release_date'].dt.day
data['month'] = data['track_album_release_date'].dt.month
data['year'] = data['track_album_release_date'].dt.year

data = data.drop(columns=['track_album_release_date'])

In [ ]:
# Rremove unnecessary column 
data = data.drop(columns=['track_id', 'track_album_id', 'playlist_id'])
data.head(3)

,track_name,track_artist,track_popularity,track_album_name,playlist_name,playlist_genre,playlist_subgenre,danceability,energy,key,...,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,mood,date,month,year
0,I Don't Care (with Justin Bieber) - Loud Luxur...,Ed Sheeran,66,I Don't Care (with Justin Bieber) [Loud Luxury...,Pop Remix,pop,dance pop,0.748,0.916,6,...,0.102000,0.000000,0.0653,0.5180,122.036,194754,Happy,14,6,2019
1,Memories - Dillon Francis Remix,Maroon 5,67,Memories (Dillon Francis Remix),Pop Remix,pop,dance pop,0.726,0.815,11,...,0.072400,0.004210,0.3570,0.6930,99.972,162600,Happy,13,12,2019
2,All the Time - Don Diablo Remix,Zara Larsson,70,All the Time (Don Diablo Remix),Pop Remix,pop,dance pop,0.675,0.931,1,...,0.079400,0.000023,0.1100,0.6130,124.008,176616,Happy,5,7,2019
3,Call You Mine - Keanu Silva Remix,The Chainsmokers,60,Call You Mine - The Remixes,Pop Remix,pop,dance pop,0.718,0.930,7,...,0.028700,0.000009,0.2040,0.2770,121.956,169093,Happy,19,7,2019
4,Someone You Loved - Future Humans Remix,Lewis Capaldi,69,Someone You Loved (Future Humans Remix),Pop Remix,pop,dance pop,0.650,0.833,1,...,0.080300,0.000000,0.0833,0.7250,123.976,189052,Sad,5,3,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32828,City Of Lights - Official Radio Edit,Lush & Simon,42,City Of Lights (Vocal Mix),♥ EDM LOVE 2020,edm,progressive electro house,0.428,0.922,2,...,0.076600,0.000000,0.0668,0.2100,128.170,204375,Sad,28,4,2014
32829,Closer - Sultan & Ned Shepard Remix,Tegan and Sara,20,Closer Remixed,♥ EDM LOVE 2020,edm,progressive electro house,0.522,0.786,0,...,0.001710,0.004270,0.3750,0.4000,128.041,353120,Sad,8,3,2013
32830,Sweet Surrender - Radio Edit,Starkillers,14,Sweet Surrender (Radio Edit),♥ EDM LOVE 2020,edm,progressive electro house,0.529,0.821,6,...,0.108000,0.000001,0.1500,0.4360,127.989,210112,Sad,21,4,2014
32831,Only For You - Maor Levi Remix,Mat Zo,15,Only For You (Remixes),♥ EDM LOVE 2020,edm,progressive electro house,0.626,0.888,2,...,0.007920,0.127000,0.3430,0.3080,128.008,367432,Sad,1,1,2014


In [93]:
# # Now start data visualization for understanign trand
# import seaborn as sns
# import matplotlib.pyplot as plt

# plt.figure(figsize=(10,8))
# sns.heatmap(data, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
# plt.title("Correlation Heatmap")
# plt.show()